In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

2024-02-26 13:38:01.074725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-26 13:38:02.619941: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 13:38:03.082809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-26 13:38:03.082855: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
# Load your dataset
df = pd.read_csv('updated_Data_till_T0_2-75_durga_oct17.csv')

# Shuffle the DataFrame
data = df.sample(frac=1, random_state=42)

# Select the first 100 rows for training and testing
data = data[:10_000]

In [3]:
df

,n1,l1,j1,n2,l2,j2,n3,l3,j3,LP,Te,Z,A,Rfq,T0
0,6,2,1.5,7,2,1.5,8,3,2.5,0.0083,0.0865,55,133,259.3814,2.0496
1,6,2,1.5,7,2,1.5,8,3,2.5,0.0083,0.1038,55,133,259.3814,2.0493
2,6,2,1.5,7,2,1.5,8,3,2.5,0.0083,0.1211,55,133,259.3814,2.0481
3,6,2,1.5,7,2,1.5,8,3,2.5,0.0083,0.1385,55,133,259.3814,2.0451
4,6,2,1.5,7,2,1.5,8,3,2.5,0.0083,0.1558,55,133,259.3814,2.0394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676156,12,3,3.5,13,1,1.5,14,2,2.5,1.0000,0.9519,11,23,75306.6725,4.9398
4676157,12,3,3.5,13,1,1.5,14,2,2.5,1.0000,0.9692,11,23,75306.6725,4.9395
4676158,12,3,3.5,13,1,1.5,14,2,2.5,1.0000,0.9865,11,23,75306.6725,4.9392
4676159,12,3,3.5,13,1,1.5,14,2,2.5,1.0000,1.0038,11,23,75306.6725,4.9390


In [28]:
np.min(df.Rfq.values)*10e7,np.max(df.Rfq.values)*10e7

(148430000.0, 18552832640000.0)

In [24]:
np.min(df.LP.values)*300

2.49

In [9]:
np.max(df.n2.values)

13

In [10]:
np.max(df.n3.values)

14

In [ ]:
# Define input features and target variables
input_features = ['n1', 'l1', 'j1', 'LP', 'Te', 'Z', 'A', 'Rfq', 'T0' ]
target_variables = ['n2', 'l2', 'j2', 'n3', 'l3', 'j3']


In [ ]:
# Split the dataset into input (X) and target (y) variables
X = data[input_features]
y = data[target_variables]

# Normalize input features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Define the ANN model as a function
def create_model(learning_rate=0.001, num_units=64, activation='relu'):
    model = models.Sequential([
        layers.Dense(num_units, activation=activation, input_shape=(9,)),
        layers.Dense(num_units, activation=activation),
        layers.Dense(6)  # Output layer with 6 units for n2, l2, j2, n3, l3, j3
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

In [ ]:
# Define a function to perform manual hyperparameter tuning
def manual_hyperparameter_tuning(learning_rate_candidates, num_units_candidates, activation_candidates):
    best_mae = float('inf')
    best_hyperparameters = None
    best_history = None

    for lr in learning_rate_candidates:
        for num_units in num_units_candidates:
            for activation in activation_candidates:
                model = create_model(learning_rate=lr, num_units=num_units, activation=activation)
                history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=0)
                y_pred = model.predict(X_test)
                mae = mean_absolute_error(y_test, y_pred)

                if mae < best_mae:
                    best_mae = mae
                    best_hyperparameters = {'learning_rate': lr, 'num_units': num_units, 'activation': activation}
                    best_history = history
                    best_model = model  # Store the best model

    return best_hyperparameters, best_history ,best_model ,best_mae

# Define candidate hyperparameter values
learning_rate_candidates = [0.001, 0.01, 0.1]
num_units_candidates = [32, 64, 128]
activation_candidates = ['relu', 'tanh']


In [ ]:
# Perform manual hyperparameter tuning
best_hyperparameters, best_history, best_model,best_mae = manual_hyperparameter_tuning(learning_rate_candidates, num_units_candidates, activation_candidates)

In [ ]:
best_hyperparameters

In [ ]:
tr_loss =  best_history.history['loss']
vl_loss =  best_history.history['val_loss']
tr_mae = best_history.history['mean_absolute_error']
vl_mae = best_history.history['val_mean_absolute_error']

In [ ]:
np.min(tr_loss),np.min(vl_loss),np.min(tr_mae),np.min(vl_mae)

In [ ]:
np.min(tr_loss),np.min(vl_loss),np.min(tr_mae),np.min(vl_mae)

In [ ]:
tt = np.array(best_history.history['val_mean_absolute_error'])/np.array(best_history.history['mean_absolute_error'])

In [ ]:
best_mae

In [ ]:
plt.plot(np.linspace(0,100,100),tt)

In [ ]:
# Plot learning curves
plt.figure(figsize=(6,4))

# Plot training & validation loss values
plt.subplot(1, 2, 1)
plt.plot(best_history.history['loss'])
plt.plot(best_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation MAE values
plt.subplot(1, 2, 2)
plt.plot(best_history.history['mean_absolute_error'])
plt.plot(best_history.history['val_mean_absolute_error'])
plt.title('Mean Absolute Error (MAE)')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
# Plot results for n2, l2, j2, n3, l3, j3 separately
y_pred = best_model.predict(X_test)
y_test = pd.DataFrame(y_test, columns=target_variables)  # Define y_test if it's not defined

plt.figure(figsize=(15, 10))
for i, target in enumerate(target_variables):
    plt.subplot(2, 3, i+1)
    plt.scatter(y_test[target], y_pred[:, i])
    plt.xlabel(f'Actual {target}')
    plt.ylabel(f'Predicted {target}')
    plt.title(f'{target} Prediction')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the absolute errors
errors = y_test.values - y_pred

# Plot individual error distributions for each target variable
plt.figure(figsize=(15, 10))
for i, target in enumerate(target_variables):
    plt.subplot(2, 3, i + 1)
    plt.hist(errors[:, i], bins=50, edgecolor='k')
    plt.title(f'Error Distribution for {target}')
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    plt.grid(True)

plt.tight_layout()
plt.show()